In [42]:
import os, sys
import chainer
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from bdg.datasets import BridgeSegPatch

data = BridgeSegPatch(data_file='total_dataset_patches_100', dataset_dir='/home/dokutagero/repos/bridgedegradationseg/dataset')

train_data, test_data = chainer.datasets.split_dataset_random(data, int(len(data)*0.8), seed=10)

train_iter = chainer.iterators.MultiprocessIterator(train_data, batch_size=1,
                                                    repeat=False, shuffle=True, n_processes=4)

test_iter = chainer.iterators.MultiprocessIterator(test_data, batch_size=1,
                                                    repeat=False, shuffle=True, n_processes=4)
counter=0
positive_train=0
print('elements in train set: ', len(train_data))
for i in range(len(train_data)):
    batch = next(train_iter)
    for elem in batch:
        counter+=1
        if elem[1] == 1:
            positive_train+=1

            
counter=0
positive_test=0
print('elements in validation set: ', len(test_data))
for i in range(len(test_data)):
    batch = next(test_iter)
    for elem in batch:
        counter+=1
        if elem[1] == 1:
            positive_test+=1

# for i in range(50):
#     im,lbl = next(train_iter)[0]
#     im = im.transpose(2, 1, 0)
#     #lbl = lbl.transpose(2,1,0)
#     #print(lbl.shape)
#     #image = Image.fromarray(im.astype(np.uint8))
#     #%matplotlib inline
#     # print(lbl.shape)
#     # print(len(np.where(lbl == 1)[0]))
#     # print(np.sum(lbl))
#     plt.figure()
#     plt.imshow(lbl)


elements in train set:  14808
elements in validation set:  3702


In [43]:
print('positive in train: ' , positive_train)
print('train size: ', len(train_data))
print('positive in test: ', positive_test)
print('test size: ', len(test_data))


positive in train:  1306
train size:  14808
positive in test:  358
test size:  3702


In [44]:
# %load examples/alex/train_alex.py
import chainer
import os
import bdg.datasets as datasets
from bdg.models.alexnet import Alex
from bdg.models.alexnet import copy_model
from chainer.links.caffe import CaffeFunction
import pickle as pkl

model_path = "/home/dokutagero/repos/bridgedegradationseg/bdg/models/"
caffe_model = os.path.join(model_path, 'bvlc_alexnet.caffemodel')
chainer_model = os.path.join(model_path, 'alexnet_chainer.pkl')

if os.path.isfile(caffe_model):
    print("Caffe model downloaded")

if not os.path.isfile(chainer_model):
    print("Creating chainer model...")
    alexnet = CaffeFunction(caffe_model)
    with open(chainer_model, 'wb') as modelfile:
        pkl.dump(alexnet, modelfile)
else:
    with open(chainer_model, 'rb') as modelfile:
        alexnet = pkl.load(modelfile)


predictor = Alex(in_channels=3, nclass=2)


train_iter = chainer.iterators.MultiprocessIterator(train_data, batch_size=32,
                                                    repeat=True, shuffle=True, n_processes=4)
test_iter = chainer.iterators.MultiprocessIterator(test_data, batch_size=32,
                                                    repeat=False, shuffle=False, n_processes=4)

# Forward for loading fully connected layers
res = 0
b = next(test_iter)
b[0][0].shape
#img = chainer.cuda.cupy.asarray(np.expand_dims(b[0][0], axis=0), dtype=np.float32)
res = predictor(np.expand_dims(b[0][0], axis=0))

copy_model(alexnet, model)
model = chainer.links.Classifier(predictor)

optimizer = chainer.optimizers.SGD(lr=0.001)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(rate=0.0005))

data = datasets.BridgeSegPatch(data_file='total_dataset_patches_100', dataset_dir='/home/dokutagero/repos/bridgedegradationseg/dataset')
train_data, test_data = chainer.datasets.split_dataset_random(data, int(len(data)*0.8), seed=10)
print('Training set size: ', len(train_data))
print('Testing set size: ', len(test_data))


updater = chainer.training.StandardUpdater(train_iter, optimizer, device=0
                                           )                                          #loss_func=chainer.functions.sigmoid_cross_entropy)

trainer = chainer.training.Trainer(updater, stop_trigger=(100, 'epoch'), out='alex_out')
# trainer.extend(chainer.training.extensions.Evaluator(train_iter, model, device=0))
trainer.extend(chainer.training.extensions.Evaluator(test_iter, model, device=0))
trainer.extend(chainer.training.extensions.LogReport())
trainer.extend(chainer.training.extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy']))
# trainer.extend(chainer.training.extensions.ProgressBar())
trainer.extend(chainer.training.extensions.PlotReport(['main/loss', 'validation/main/loss'], 'epoch', file_name='loss.png'))
trainer.extend(chainer.training.extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], 'epoch', file_name='acc.png'))
trainer.extend(chainer.training.extensions.snapshot_object(predictor, filename="model_it_{.updater.iteration}"), chainer.training.triggers.IntervalTrigger(5, 'epoch'))
trainer.run()


Caffe model downloaded
Training set size:  14808
Testing set size:  3702
epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy


KeyboardInterrupt: 

In [38]:
test_iter = chainer.iterators.MultiprocessIterator(test_data, batch_size=32,
                                                    repeat=False, shuffle=False, n_processes=4)
res = 0
b = next(test_iter)
b[0][0].shape
img = chainer.cuda.cupy.asarray(np.expand_dims(b[0][0], axis=0), dtype=np.float32)
res = predictor(img)


In [39]:
res

variable([[ 0.95840234, -0.98283112]])